In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import dependancies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Import dataset

In [ ]:
train= pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test= pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.describe().T

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set a custom style and color palette
sns.set_theme(style="whitegrid")
colors = sns.color_palette("viridis", len(train['label'].unique()))

plt.figure(figsize=(10, 6))
ax = sns.countplot(
    x=train['label'],
    palette=colors,
    order=sorted(train['label'].unique())  # Ensure labels are in order
)

for p in ax.patches:
    ax.annotate(
        f'{int(p.get_height())}', 
        (p.get_x() + p.get_width() / 2., p.get_height()), 
        ha='center', va='center', 
        xytext=(0, 5), textcoords='offset points', 
        fontsize=10, color='black'
    )

plt.title('Unique Distribution of Digits in the Dataset', fontsize=14, fontweight='bold')
plt.xlabel('Digit Label', fontsize=12)
plt.ylabel('Count', fontsize=12)

plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 5), constrained_layout=True)

# Iterate through the first 5 rows of the dataset
for i, ax in enumerate(axes):
    ax.imshow(train.iloc[i, 1:].values.reshape(28, 28), cmap='gray')
    ax.set_title(f"Label: {train['label'][i]}", fontsize=12, fontweight='bold', color='darkblue')
    ax.axis('off')  # Hide axes for a cleaner look

# Add an overall title for the figure
fig.suptitle("Sample Digits from the Dataset", fontsize=16, fontweight='bold', color='darkgreen')

# Show the plot
plt.show()

## Data Preprocessing

In [ ]:
# Separate features and labels
X = train.drop('label', axis=1).values / 255.0  # Normalize pixel values
y = train['label'].values

X = X.reshape(-1, 28, 28, 1)  
y = tf.keras.utils.to_categorical(y, 10)  

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

## CNN Modeling

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    # Input Layer
    Input(shape=(28, 28, 1)),

    # First Convolutional Block
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Second Convolutional Block
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    # Third Convolutional Block
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.5),

    # Fully Connected Layers
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

# Print model summary
model.summary()

## Compiling modeling

In [ ]:
# Compile the model
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

## Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=12,      
    zoom_range=0.15,         
    width_shift_range=0.15,  
    height_shift_range=0.15  
)

datagen.fit(X_train)

## Train modelling

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64), 
    validation_data=(X_val, y_val),              
    epochs=30,                                     
    callbacks=[early_stopping, reduce_lr]          
)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6), constrained_layout=True)

# Plot training and validation accuracy
axes[0].plot(history.history['accuracy'], label='Train Accuracy', color='dodgerblue', linestyle='-', marker='o', markersize=6)
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', color='orange', linestyle='--', marker='s', markersize=6)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epochs', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Plot training and validation loss
axes[1].plot(history.history['loss'], label='Train Loss', color='green', linestyle='-', marker='x', markersize=6)
axes[1].plot(history.history['val_loss'], label='Validation Loss', color='red', linestyle='--', marker='^', markersize=6)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epochs', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].legend()

# Show the plot
plt.show()

## Model Evaluation

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)

# Print the validation accuracy
print(f"Validation Accuracy: {val_accuracy:.4f}")

## Making predictions

In [ ]:

test_normalized = test / 255.0
test_normalized = test_normalized.values.reshape(-1, 28, 28, 1)

predictions = np.argmax(model.predict(test_normalized), axis=1)

submission = pd.DataFrame({'ImageId': range(1, len(predictions) + 1), 'Label': predictions})
submission.to_csv('submission.csv', index=False)

print("Submission file saved as 'submission.csv'")